<a href="https://colab.research.google.com/github/VenkteshV/Question_duplicate_detection/blob/main/Week6/TopicRank_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet keybert
!pip install --quiet git+https://github.com/boudinfl/pke.git

In [ ]:
import pandas as pd
import spacy
import pke
from tqdm import tqdm
from spacy.matcher import Matcher
import time
import numpy as np
from keybert import KeyBERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Boilerplpate for working with POS tags

In [ ]:
patterns = [
        [{'POS': 'PROPN'}, {'POS': 'VERB'}, {'POS': 'VERB'}],
        [{'POS': 'NOUN'}, {'POS': 'VERB'}, {'POS': 'NOUN'}],
        [{'POS': 'VERB'}, {'POS': 'NOUN'}],
        [{'POS': 'ADJ'}, {'POS': 'ADJ'}, {'POS': 'NOUN'}],  
        [{'POS': 'NOUN'}, {'POS': 'VERB'}],
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'NOUN'}],
        [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
        [{'POS': 'ADJ'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'ADV'}, {'POS': 'PROPN'}],
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'VERB'}],
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],
        [{'POS': 'NOUN'}, {'POS': 'NOUN'}],
        [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
        [{'POS': 'PROPN'}, {'POS': 'ADP'}, {'POS': 'PROPN'}],
        [{'POS': 'PROPN'}, {'POS': 'ADJ'}, {'POS': 'NOUN'}],
        [{'POS': 'PROPN'}, {'POS': 'VERB'}, {'POS': 'NOUN'}],
        [{'POS': 'NOUN'}, {'POS': 'ADP'}, {'POS': 'NOUN'}],
        [{'POS': 'PROPN'}, {'POS': 'NOUN'}, {'POS': 'PROPN'}],
        [{'POS': 'VERB'}, {'POS': 'ADV'}],
        [{'POS': 'PROPN'}, {'POS': 'NOUN'}],
        ]

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

matcher.add("MyMatcher", patterns)

doc = nlp('''
  My pattern matcher. We’ll define a benchmark function that takes in our corpus and a boolean for shuffling or not our data. For each extractor, it calls the extract_keywords_from_corpus function, which returns a dictionary containing the result of that extractor. We store that value in a list.
  For each algorithm in the list, we compute
  average number of extracted keywords
  average number of matched keywords
  compute a score that takes into account the average number of matches found divided by how much time it took to perform the operation
  '''
)

matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(string_id, start, end, span.text)

MyMatcher 2 4 pattern matcher
MyMatcher 9 11 benchmark function
MyMatcher 39 41 dictionary containing
MyMatcher 66 68 average number
MyMatcher 69 71 extracted keywords
MyMatcher 72 74 average number
MyMatcher 75 77 matched keywords
MyMatcher 86 88 average number
MyMatcher 87 90 number of matches
MyMatcher 89 91 matches found
MyMatcher 94 96 much time


### Using the Topic_Rank_Extractor

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Literature Review/Dataset versions/SSDS/SSDS_base_dataset_10kquestions")
df = df.drop("Unnamed: 0", axis= 1)

In [ ]:
# df = df[:10]

In [ ]:
questions_ls = df['question']

In [ ]:
df.head()

,question_id,question,syllabus_id,answer,question_type
0,1523485,the first method used for determining the actu...,518161,Swedish geologist Gerard De Geer,Single Choice Question
1,833487,when we subtract a number from itself we get,86295,zero,Single Choice Question
2,2057553,what orbitals has two spherical nodes,697667,6f,Single Choice Question
3,1360683,the reagent used to step up in the homologous ...,449190,Diazomethane,Single Choice Question
4,849902,the shortcut key to delete a slide,5118,Ctrl X,Single Choice Question


In [ ]:
bert = KeyBERT()
nlp = spacy.load("en_core_web_sm")

In [ ]:
# def keybert_extractor(text):
#     keywords = bert.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words="english", top_n=1)
#     results = []
#     for scored_keywords in keywords:
#         for keyword in scored_keywords:
#             if isinstance(keyword, str):
#                 results.append(keyword)
#     return results 

In [ ]:
def topic_rank_extractor(text):
    extractor = pke.unsupervised.TopicRank()
    extractor.load_document(text, language='en')
    pos = {'NOUN', 'PROPN', 'ADJ', 'ADV'}
    extractor.candidate_selection(pos=pos)
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=5)
    results = []
    for scored_keywords in keyphrases:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results

In [ ]:
topic_rank_extractor("We are participating in a hackathon by StarkNet to learn Cairo and build awesome project with it")

['starknet', 'cairo', 'hackathon', 'awesome project']

In [ ]:
corpus_kws = {}
for idx, text in tqdm(enumerate(questions_ls)):
    corpus_kws[idx] = topic_rank_extractor(text)

10000it [2:26:27,  1.14it/s]


In [ ]:
corpus_kws_df = pd.DataFrame(pd.Series(corpus_kws), columns=["keywords"])

In [ ]:
df = pd.concat([df, corpus_kws_df], axis=1)

In [ ]:
df.to_csv('/content/drive/MyDrive/Literature Review/Dataset versions/SSDS/Added-extracted_keywords.csv')